In [13]:
!pip install torch==2.1.2
!pip install torchvision==0.16.2
!pip install transformers==4.40.0
!pip install sentencepiece==0.1.99
!pip install flash-attn

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [14]:
!pip install bitsandbytes
!pip install -U accelerate

In [15]:
!pip install pyngrok

!killall ngrok

/bin/bash: line 1: killall: command not found


In [16]:
!pip install --upgrade pip setuptools wheel
!pip install ninja cmake
!pip install git+https://github.com/HazyResearch/flash-attention --no-build-isolation
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Verify flash-attn installation
!pip show flash_attn


from transformers import AutoModel, AutoTokenizer
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6-int4', trust_remote_code=True, force_download=True)
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6-int4', trust_remote_code=True, force_download=True)
model.eval()


  Cloning https://github.com/HazyResearch/flash-attention to /tmp/pip-req-build-xlkmey7u
  Running command git clone --filter=blob:none --quiet https://github.com/HazyResearch/flash-attention /tmp/pip-req-build-xlkmey7u
  Resolved https://github.com/HazyResearch/flash-attention to commit c1d146cbd5becd9e33634b1310c2d27a49c7e862
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
Looking in indexes: https://download.pytorch.org/whl/cu118
Name: flash-attn
Version: 2.6.3
Summary: Flash Attention: Fast and Memory-Efficient Exact Attention
Home-page: https://github.com/Dao-AILab/flash-attention
Author: Tri Dao
Author-email: tri@tridao.me
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: einops, torch
Required-by: 


config.json:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/3.28k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.56k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.64k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


MiniCPMV(
  (llm): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(151666, 3584)
      (layers): ModuleList(
        (0-27): 28 x Qwen2DecoderLayer(
          (self_attn): Qwen2SdpaAttention(
            (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
            (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
            (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
            (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
            (rotary_emb): Qwen2RotaryEmbedding()
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
            (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
            (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm()
          (post_attention_lay

In [17]:
from flask import Flask, request, jsonify
import threading
import io
import time

app = Flask(__name__)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.form
    prompt = data.get("user_input", "")

    if not prompt:
        return jsonify({"error": "No prompt provided"}), 400

    msgs = [{'role': 'user', 'content': [prompt]}]

    response = model.chat(
        image=None,
        msgs=msgs,
        tokenizer=tokenizer
    )
    print(response)
    return jsonify({"response": response})

@app.route("/image", methods=["POST"])
def image_to_text():
    data = request.form
    prompt = data.get("user_input", "")
    image_data = request.files.get('image')
    if not prompt:
        return jsonify({"error": "No prompt provided"}), 400

    image = Image.open(io.BytesIO(image_data.read())).convert('RGB')
    msgs = [{'role': 'user', 'content': [image, prompt]}]

    response = model.chat(
        image=None,
        msgs=msgs,
        tokenizer=tokenizer
    )
    return jsonify({"response": response})

server_thread = None

def run_app():
    global server_thread
    if server_thread and server_thread.is_alive():
        print("Stopping previous server...")
        time.sleep(1)

    server_thread = threading.Thread(target=lambda: app.run(host="localhost", port=port, debug=True, use_reloader=False))
    server_thread.start()
    print("Server started on port 5000")


In [18]:
from pyngrok import ngrok
port=5000
ngrok.set_auth_token("2nZDykSbW5YtWlLhXAqfzARWZuQ_5LrqyHG7nSaNWC27BJkKW")

public_url = ngrok.connect(port)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://3177-34-145-10-28.ngrok-free.app" -> "http://localhost:5000"


In [19]:
run_app()

Server started on port 5000
 * Serving Flask app '__main__'
 * Debug mode: on


preprocessor_config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

processing_minicpmv.py:   0%|          | 0.00/9.96k [00:00<?, ?B/s]

image_processing_minicpmv.py:   0%|          | 0.00/16.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6-int4:
- image_processing_minicpmv.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6-int4:
- processing_minicpmv.py
- image_processing_minicpmv.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
